# Токсичность комметариев

### Описание проекта
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества F1 не меньше 0.75.

Столбец text в нём содержит текст комментария, а toxic — целевой признак.

### Содержание

1. [Изучение и обработка данных](#start)
2. [Обучение моделей](#model_training)       
3. [Лучшая модель](#the_best_of_the_best)  

In [1]:
import pandas as pd
import numpy as np
import time
import random
from IPython.display import display
import spacy
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import transformers
from tqdm import notebook
from tqdm import tqdm

import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('wordnet')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 1. Изучение и обработка данных<a id="start"></a>

In [24]:
try:
    toxic_comments = pd.read_csv('D:/Users/Иван/Downloads/Токсичность комметариев/toxic_comments.csv')
except:
    toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
toxic_comments.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [25]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


В тексте наблюдаем верхний и нижний регистр шрифта, не текстовые символы и прочий мусор, от которого для корректного обучения необходимо избавиться. Также в целевом признаке на десяти текстах видим дисбаланс, что вызывает подозрения. С проверки дисбаланса и начнём:

In [26]:
# проверим баланс
toxic_comments['toxic'].value_counts(dropna=False)

toxic
0    143106
1     16186
Name: count, dtype: int64

Видим, что дисбаланс на целом кортеже имеет соотношение 9:1, что не очень хорошо.

In [27]:
# Установим seed для воспроизводимости
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Данные сбалансировали, теперь приступим к правильному виду текста, для этого лемматизируем его, приведём к нижнему регистру:

In [28]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])# лемматизируем по словам и объединяем в строку
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

In [30]:
# Инициализация BERT токенизатора
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Функция для предварительной обработки текста для BERT
def preprocess_for_bert(texts, labels, max_length=128):
    input_ids = []
    attention_masks = []
    
    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,                     
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels.values)
    
    return input_ids, attention_masks, labels

Далее леммитизируем данные:

In [31]:
toxic_comments['lemmatized'] = toxic_comments['text'].apply(lemmatize_text)
toxic_comments = toxic_comments.drop(['text'], axis=1)#удалим дублирующую колонку
toxic_comments.head(10)

,Unnamed: 0,toxic,lemmatized
0,0,0,explanation why the edits made under my userna...
1,1,0,d aww he match this background colour i m seem...
2,2,0,hey man i m really not trying to edit war it s...
3,3,0,more i can t make any real suggestion on impro...
4,4,0,you sir are my hero any chance you remember wh...
5,5,0,congratulation from me a well use the tool wel...
6,6,1,cocksucker before you piss around on my work
7,7,0,your vandalism to the matt shirvington article...
8,8,0,sorry if the word nonsense wa offensive to you...
9,9,0,alignment on this subject and which are contra...


Далее выделим целевой признак для дальнейшего разбиения:

In [32]:
features = toxic_comments['lemmatized']
target = toxic_comments['toxic']

Разобьем выборку по отношению 60:20:20. В связи с недостаточностью памяти valid и test выборки сформируем из 5000 строк, так как после балансировки train число строк сильно уменьшится.

In [33]:
#из выборки выбираем 60% под трейн, которые дальше семплируем из оставшейся выборки выделяем 5000 записей, 
# которые делим на valid и test
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=5000, 
                                                                              random_state=12345 )

features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test, test_size=0.5, 
                                                                              random_state=12345 )
print(features_train.shape, features_valid.shape, features_test.shape)
print(target_train.shape, target_valid.shape, target_test.shape)

(95575,) (2500,) (2500,)
(95575,) (2500,) (2500,)


Разбив выборки на train/valid/test приступаем к даунсемплингу нулевого класса

In [34]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

In [35]:
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.2)
target_downsampled.value_counts()

toxic
0    17185
1     9652
Name: count, dtype: int64

In [36]:
# Подготовка данных для BERT

train_texts = toxic_comments.loc[features_downsampled.index, 'lemmatized']
train_labels = target_downsampled

valid_texts = toxic_comments.loc[features_valid.index, 'lemmatized']
valid_labels = target_valid

test_texts = toxic_comments.loc[features_test.index, 'lemmatized']
test_labels = target_test

In [37]:
# Преобразование данных для BERT
train_inputs, train_masks, train_labels_bert = preprocess_for_bert(train_texts, train_labels)
val_inputs, val_masks, val_labels_bert = preprocess_for_bert(valid_texts, valid_labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\user\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [38]:
# Создание DataLoader для BERT
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels_bert)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(val_inputs, val_masks, val_labels_bert)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

Загрузим стоп-слова и применим модель TfidfVectorizer, обучив её на наших данных:

In [39]:
# Загрузка стоп-слов (это нужно сделать один раз)
nltk.download('stopwords')
stopwords = list(nltk_stopwords.words('english'))  # Преобразуем в список

# Инициализация TfidfVectorizer с заданными стоп-словами
tf_idf = TfidfVectorizer(stop_words=stopwords)

# Обучение и преобразование обучающего набора
features_train = tf_idf.fit_transform(features_downsampled).toarray()

# Преобразование валидационного и тестового наборов
features_valid = tf_idf.transform(features_valid).toarray()
features_test = tf_idf.transform(features_test).toarray()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Вывод:**

1. Выявили и устранили дисбаланс классов;
2. Подготовили текст к обучению;
3. Разбили выборки на тренировочную, валидационную и тестовую;
4. Применили модель TfidfVectorizer для обучения на данных.

## 2. Обучение моделей<a id="model_training"></a>

### Random Forest Classifier¶

In [18]:
model_rfc =  RandomForestClassifier(random_state=12345, n_estimators=100, max_depth=8)
model_rfc.fit(features_train, target_downsampled)

RandomForestClassifier(max_depth=8, random_state=12345)

In [19]:
predicted_valid_rfc = model_rfc.predict(features_valid)
f1_rfc_valid = f1_score(target_valid, predicted_valid_rfc)

print('F1 на валидационной выборке', f1_rfc_valid)

F1 на валидационной выборке 0.007434944237918215


### CatBoostRegressor

In [20]:
#for estimator in range(10, 101, 10):
#    model =  CatBoostClassifier(random_state=12345, n_estimators=estimator)
#    model.fit(features_train, target_train)

In [40]:
model_cbr =  CatBoostClassifier(random_state=12345, n_estimators=100)
model_cbr.fit(features_train, target_downsampled)

Learning rate set to 0.34674
0:	learn: 0.5775899	total: 195ms	remaining: 19.4s
1:	learn: 0.5205743	total: 396ms	remaining: 19.4s
2:	learn: 0.4842611	total: 595ms	remaining: 19.2s
3:	learn: 0.4639008	total: 794ms	remaining: 19.1s
4:	learn: 0.4465827	total: 998ms	remaining: 19s
5:	learn: 0.4346294	total: 1.2s	remaining: 18.8s
6:	learn: 0.4202655	total: 1.4s	remaining: 18.5s
7:	learn: 0.4131415	total: 1.6s	remaining: 18.4s
8:	learn: 0.4053250	total: 1.8s	remaining: 18.2s
9:	learn: 0.3976346	total: 2.01s	remaining: 18.1s
10:	learn: 0.3901463	total: 2.2s	remaining: 17.8s
11:	learn: 0.3849021	total: 2.41s	remaining: 17.7s
12:	learn: 0.3797039	total: 2.62s	remaining: 17.5s
13:	learn: 0.3754023	total: 2.82s	remaining: 17.3s
14:	learn: 0.3713793	total: 3.02s	remaining: 17.1s
15:	learn: 0.3642000	total: 3.22s	remaining: 16.9s
16:	learn: 0.3607108	total: 3.42s	remaining: 16.7s
17:	learn: 0.3572985	total: 3.62s	remaining: 16.5s
18:	learn: 0.3536664	total: 3.82s	remaining: 16.3s
19:	learn: 0.349655

In [41]:
predicted_valid_cbr = model_cbr.predict(features_valid)
f1_cbr_valid = f1_score(target_valid, predicted_valid_cbr)

print('F1 на валидационной выборке', f1_cbr_valid)

F1 на валидационной выборке 0.7587476979742173


### BERT Classifier

In [42]:
# Загрузка модели BERT
model_bert = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
# Настройка оптимизатора
optimizer = AdamW(model_bert.parameters(),
                lr=2e-5,
                eps=1e-8)

C:\Users\user\anaconda3\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [44]:
# Настройка планировщика обучения
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                          num_warmup_steps=0,
                                          num_training_steps=total_steps)

In [45]:
# Функция для вычисления точности
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [46]:
# Обучение модели BERT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_bert.to(device)

for epoch_i in range(0, epochs):
    print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
    t0 = time.time()
    total_loss = 0
    model_bert.train()
    
    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = time.strftime("%H:%M:%S", time.gmtime(time.time() - t0))
            print(f'  Batch {step} of {len(train_dataloader)}. Elapsed: {elapsed}')
            
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        model_bert.zero_grad()        
        outputs = model_bert(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
    avg_train_loss = total_loss / len(train_dataloader)
    training_time = time.strftime("%H:%M:%S", time.gmtime(time.time() - t0))
    
    print(f"  Average training loss: {avg_train_loss:.2f}")
    print(f"  Training epoch took: {training_time}")

======== Epoch 1 / 3 ========
  Batch 40 of 839. Elapsed: 00:00:07
  Batch 80 of 839. Elapsed: 00:00:14
  Batch 120 of 839. Elapsed: 00:00:21
  Batch 160 of 839. Elapsed: 00:00:28
  Batch 200 of 839. Elapsed: 00:00:35
  Batch 240 of 839. Elapsed: 00:00:42
  Batch 280 of 839. Elapsed: 00:00:49
  Batch 320 of 839. Elapsed: 00:00:56
  Batch 360 of 839. Elapsed: 00:01:03
  Batch 400 of 839. Elapsed: 00:01:10
  Batch 440 of 839. Elapsed: 00:01:17
  Batch 480 of 839. Elapsed: 00:01:24
  Batch 520 of 839. Elapsed: 00:01:31
  Batch 560 of 839. Elapsed: 00:01:38
  Batch 600 of 839. Elapsed: 00:01:45
  Batch 640 of 839. Elapsed: 00:01:52
  Batch 680 of 839. Elapsed: 00:01:59
  Batch 720 of 839. Elapsed: 00:02:06
  Batch 760 of 839. Elapsed: 00:02:13
  Batch 800 of 839. Elapsed: 00:02:20
  Average training loss: 0.20
  Training epoch took: 00:02:27
======== Epoch 2 / 3 ========
  Batch 40 of 839. Elapsed: 00:00:07
  Batch 80 of 839. Elapsed: 00:00:14
  Batch 120 of 839. Elapsed: 00:00:21
  Batch 

In [47]:
# Оценка BERT на тестовых данных
test_inputs, test_masks, test_labels_bert = preprocess_for_bert(test_texts, test_labels)
test_dataset = TensorDataset(test_inputs, test_masks, test_labels_bert)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

model_bert.eval()
predictions, true_labels = [], []

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        outputs = model_bert(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask)
    
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    predictions.append(logits)
    true_labels.append(label_ids)

# Вычисление F1-score для BERT
flat_predictions = np.concatenate(predictions, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = np.concatenate(true_labels, axis=0)

f1_bert = f1_score(flat_true_labels, flat_predictions)
print(f'F1 BERT на тестовой выборке: {f1_bert:.4f}')

C:\Users\user\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


F1 BERT на тестовой выборке: 0.7453


### Logistic Regression

In [48]:
model_lr = LogisticRegression(solver='liblinear', random_state=12345)
model_lr.fit(features_train, target_downsampled)

LogisticRegression(random_state=12345, solver='liblinear')

In [49]:
predicted_valid_lr = model_lr.predict(features_valid)
f1_lr_valid = f1_score(target_valid, predicted_valid_lr)

print('F1 на валидационной выборке', f1_lr_valid)

F1 на валидационной выборке 0.7554744525547445


Как видно из таблицы, лучше всех себя показал CatBoostRegressor с результатом F1 0.75. Соответственно до тестовой выборки допускается именно Cat.

## 3. Лучшая модель<a id="the_best_of_the_best"></a>

In [50]:
predicted_test_cbr = model_cbr.predict(features_test)

f1_cbr_test = f1_score(target_test, predicted_test_cbr)

print('F1 на валидационной выборке', f1_cbr_test)

F1 на валидационной выборке 0.7317073170731708


**Вывод:**
- Обучить модель классифицировать комментарии на позитивные и негативные нам удалось;
- Найти и построить модель со значением метрики качества F1 не меньше 0.75 удалось;
- Лучшую модель, **CatBoostRegressor**, протестировали на тестовой выборке, получили результат метрики качества **F1 0.73**.